# Analysis with Coffea

<h2>Authors</h2>
<b>Notebook by:</b> Mat Adamec (<i>UNL</i>)
<br/>
<br/>
<b>coffea:</b>
<br/>
<a href="https://doi.org/10.5281/zenodo.6335374"><img src="https://zenodo.org/badge/DOI/10.5281/zenodo.6335374.svg" alt="DOI"></a>
<br/>
    <ul>
        <li>Lindsey Gray, Matteo Cremonesi, Bo Jayatilaka, Oliver Gutsche, Nick Smith, Allison Hall, Kevin Pedro, Maria Acosta <i>(FNAL)</i>; Andrew Melo <i>(Vanderbilt)</i>; Stefano Belforte <i>(INFN)</i>; and others</li>
        <li>In collaboration with IRIS-HEP members: Jim Pivarski <i>(Princeton)</i>, Ben Galewsky <i>(NCSA)</i>, Mark Neubauer <i>(UIUC)</i></li>
        <br/>
    </ul>
<b><a href="https://github.com/CoffeaTeam/coffea-casa">coffea-casa</a>:</b>
    <ul>
        <li>Ken Bloom, Oksana Shadura <i>(UNL)</i>; Garhan Attebury, Carl Lundstedt, Derek Weitzel <i>(UNL-HCC)</i>; Mátyás Selmeci <i>(UWM)</i>; Brian Bockelman <i>(Morgridge Institute)</i></li>
    </ul>

## What is Coffea?

Coffea stands for *Columnar Object Framework For Effective Analysis.* It contains a variety of tools which help physicists perform their analyses in a columnar fashion. By "a columnar fashion," we mean that data is contained in numpy-like arrays upon which we can perform operations without calling an explicit event loop. Coffea is built on Awkward arrays and any Awkward operation can be done in a Coffea analysis. Beyond this, Coffea's analysis features can be broken into roughly four categories:

* **NanoEvents** turns data into an Awkward array wrapped with a schema. The schema serves a variety of purposes, from enabling us to interpret data as physics objects (e.g., LorentzVectors) to handling the nesting of our fields and creating relevant physical cross-references. In short: NanoEvents makes our data act nicely in the context of a physics analysis. A schema can be made for any ntuple file.

* **Hists** permit the plotting of ROOT-like histograms with Coffea.

* **Processors** are Coffea's way of encapsulating an analysis in a way that is deployment-neutral. Once you have a Coffea analysis, you can throw it into a processor and use any of a variety of executors (e.g. Dask, Parsl, Spark) to chunk it up and run it across distributed workers. This makes scale-out simple and dynamic on the user's end.

* **Lookup tools** are available in Coffea for any corrections that need to be made to physics data. These tools read a variety of correction file formats and turn them into lookup tables.

We will go through the first three points within this tutorial. As corrections tend to be experiment-specific, they are outside our scope, but the coffea docs offer some [examples](https://coffeateam.github.io/coffea/notebooks/applying_corrections.html) from a CMS perspective.

For the purposes of this tutorial we will be using a data sample converted to a generic ntuple format from a [Powheg+Pythia ttbar dataset](https://opendata.cern.ch/record/19981). It contains 40600 events at 1.3 GB.

## A Motivating Comparison

Placeholder. Showcase coffea vs. event loop to motivate why we'd want to use it.
* Find a suitable analysis for this.
* Maybe show the analysis we'll build through the tutorial, just to also show "where we'll end up?"
* If we do make a comparison here, then imports/NanoAODSchema warning messages will be here, so they remain here for now.

In [ ]:
from coffea.nanoevents import NanoEventsFactory, BaseSchema
from coffea import hist
import awkward as ak
import numpy as np
import uproot

# NanoEvents will try to build crossrefs that aren't in our file! Silence this as it's irrelevant for our purposes.
NanoAODSchema.warn_missing_crossrefs = False

## **Preliminary**: Data with Awkward

As mentioned in the overview, NanoEvents essentially bundles data with physical meaning. We input some ntuple file and we get an awkward array with some added utilities for physics. To follow a Coffea analysis, then, you need some familiarity with awkward. Awkward has a lot of great tutorials to refer to (including the one which preceded this talk), but this preliminary section is intended to show the necessary basics.

NanoEvents uses uproot to access data. The data is accessed lazily, so it is not instantiated until it is used. As our file is in an ntuple format, we can open it with the <code>BaseSchema</code> for now for the purposes of exploration. Later, we will use a schema to demonstrate the real power of NanoEvents.

In [15]:
events = NanoEventsFactory.from_root('https://xrootd-local.unl.edu:1094/store/user/AGC/FCBEF10A-19D4-E511-83BF-E41D2D08DCA0_merged.root', schemaclass=BaseSchema, treepath='events').events()

What does our data look like? Well, it's an awkward array. Each entry in the awkward array corresponds to an event, and each event has various fields that stem from our data. It is a simple command to examine the fields:

In [23]:
events.fields

['numberelectron',
 'nelectron_e',
 'electron_e',
 'nelectron_pt',
 'electron_pt',
 'nelectron_px',
 'electron_px',
 'nelectron_py',
 'electron_py',
 'nelectron_pz',
 'electron_pz',
 'nelectron_eta',
 'electron_eta',
 'nelectron_phi',
 'electron_phi',
 'nelectron_ch',
 'electron_ch',
 'nelectron_iso',
 'electron_iso',
 'nelectron_isLoose',
 'electron_isLoose',
 'nelectron_isMedium',
 'electron_isMedium',
 'nelectron_isTight',
 'electron_isTight',
 'nelectron_dxy',
 'electron_dxy',
 'nelectron_dz',
 'electron_dz',
 'nelectron_dxyError',
 'electron_dxyError',
 'nelectron_dzError',
 'electron_dzError',
 'numGenPart',
 'nGenPart_pt',
 'GenPart_pt',
 'nGenPart_eta',
 'GenPart_eta',
 'nGenPart_mass',
 'GenPart_mass',
 'nGenPart_pdgId',
 'GenPart_pdgId',
 'nGenPart_phi',
 'GenPart_phi',
 'nGenPart_px',
 'GenPart_px',
 'nGenPart_py',
 'GenPart_py',
 'nGenPart_pz',
 'GenPart_pz',
 'nGenPart_status',
 'GenPart_status',
 'numberjet',
 'njet_e',
 'jet_e',
 'njet_pt',
 'jet_pt',
 'njet_px',
 'jet_p

It is also fairly simple (with Awkward) to check how many events we have:

In [26]:
ak.size(events, axis=0)

40600

Of course, each field of an event might have a variable amount of objects within it. We might have 0, 1, 2, or any n number of muons in a single event. This is what is meant by "jagged" data. If we look at muons in particular:

In [28]:
events.muon_pt

<Array [[5.9, 4.44, 2.29, ... [43.6, 40.7]] type='40600 * var * float32[paramete...'>

In [31]:
events.muon_pt[0:2]

<Array [[5.9, 4.44, 2.29, 0.761], []] type='2 * var * float32[parameters={"__doc...'>

Looking at <code>events.muon_pt</code> we clearly see this jagged structure. It is an array of size 40600, but each element of the array is itself a (sub)array of variable length. The picture is clarified if we look at a subset of <code>events.muon_pt</code> such that we don't run into the print length cut-off. The first event has 4 muon_pts (and thus 4 muons) while the second event has none (it is an empty subarray).

To drive the point home, let's count up the number of muons across all of our events. <code>ak.num</code> can be used to tell us how many muons are in each event:

In [36]:
ak.num(events.muon_pt, axis=-1)

<Array [4, 0, 1, 0, 3, 4, ... 0, 7, 6, 1, 2, 2] type='40600 * int64'>

A quick note about axes in Awkward: 0 is always the shallowest, while -1 is the deepest. In other words, <code>axis=0</code> would tell us the number of subarrays (events), while <code>axis=-1</code> sums up the number of muons within each subarray. Then we can just sum up these counts to get the number of muons across all events:

In [37]:
ak.sum(ak.num(events.muon_pt, axis=-1))

74656

So the number of muons is clearly not the same as the number of events. 

Now that we know how to access data, we can manipulate it as we desire in the standard awkward way. Most cuts in columnar analysis are achieved through masking. Shortly, a mask is a Boolean array which is generated by performing a conditional on a data array. For example, if we want only muons with a p<sub>T</sub> > 10, our mask would be:

In [38]:
events.muon_pt > 10

<Array [[False, False, False, ... [True, True]] type='40600 * var * bool'>

Then, we can apply the mask to our data. This will pick out only the elements of our data which correspond to a <code>True</code>. The data and the mask thus must have the same shape up to the depth of the selection. Since we're making a selection on muons, the mask must have the exact same shape as the <code>events.muon_pt</code> data. Conversely, the shape of the output array will differ from the data and mask arrays since we want to "throw out" the data that doesn't meet our requirement.

In [46]:
'Input:', events.muon_pt, 'Output (pT > 10):', events.muon_pt[events.muon_pt > 10]

('Input:',
 <Array [[5.9, 4.44, 2.29, ... [43.6, 40.7]] type='40600 * var * float32[paramete...'>,
 'Output (pT > 10):',
 <Array [[], [], [14.1], ... [], [43.6, 40.7]] type='40600 * var * float32[parame...'>)

Note that we still have 40600 subarrays, as we still have 40600 events; we only did a selection on muons. If an event had a muon which didn't meet the cut, then that event just has an empty subarray now.

Compare the output array to our original data array. The first muons, with p<sub>T</sub> < 10, are no longer present in the array. We'd expect to have fewer muons overall. Let's take another count!

In [47]:
ak.sum(ak.num(events.muon_pt[events.muon_pt > 10]))

20504

Conversely, the set of muons whose pT is less than 10 can also be examined.

In [48]:
events.muon_pt[events.muon_pt < 10]

<Array [[5.9, 4.44, 2.29, 0.761, ... 2.64], []] type='40600 * var * float32[para...'>

Here, we see the first muons *are* present, as we'd expect. Doing some quick algebra, we'd expect this array to be 74656 - 20504 = 54152 elements in length.

In [50]:
ak.sum(ak.num(events.muon_pt[events.muon_pt < 10]))

54152

## **NanoEvents**

We now turn our attention to NanoEvents. While it's certainly possible to access and manipulate data strictly with awkward, NanoEvents schemas make data much nicer to work with. You can make a schema to work with your particular analysis, or you can use one of the existing schemas if you use common file formats (e.g., NanoAOD).

For the purposes of this tutorial, we have made a simple schema whose features we shall conflate with the default awkward behavior. Let's examine that behavior first. A good starting point is taking an inventory of our branches:

In [51]:
import uproot
branches = uproot.open("https://xrootd-local.unl.edu:1094/store/user/AGC/FCBEF10A-19D4-E511-83BF-E41D2D08DCA0_merged.root")['events']
branches.keys()

['numberelectron',
 'nelectron_e',
 'electron_e',
 'nelectron_pt',
 'electron_pt',
 'nelectron_px',
 'electron_px',
 'nelectron_py',
 'electron_py',
 'nelectron_pz',
 'electron_pz',
 'nelectron_eta',
 'electron_eta',
 'nelectron_phi',
 'electron_phi',
 'nelectron_ch',
 'electron_ch',
 'nelectron_iso',
 'electron_iso',
 'nelectron_isLoose',
 'electron_isLoose',
 'nelectron_isMedium',
 'electron_isMedium',
 'nelectron_isTight',
 'electron_isTight',
 'nelectron_dxy',
 'electron_dxy',
 'nelectron_dz',
 'electron_dz',
 'nelectron_dxyError',
 'electron_dxyError',
 'nelectron_dzError',
 'electron_dzError',
 'numGenPart',
 'nGenPart_pt',
 'GenPart_pt',
 'nGenPart_eta',
 'GenPart_eta',
 'nGenPart_mass',
 'GenPart_mass',
 'nGenPart_pdgId',
 'GenPart_pdgId',
 'nGenPart_phi',
 'GenPart_phi',
 'nGenPart_px',
 'GenPart_px',
 'nGenPart_py',
 'GenPart_py',
 'nGenPart_pz',
 'GenPart_pz',
 'nGenPart_status',
 'GenPart_status',
 'numberjet',
 'njet_e',
 'jet_e',
 'njet_pt',
 'jet_pt',
 'njet_px',
 'jet_p

That's a mouthful. Let's narrow our focus to the muon branches:

In [52]:
for branch in branches.keys():
    if 'muon' in branch:
        print(branch)

numbermuon
nmuon_e
muon_e
nmuon_pt
muon_pt
nmuon_px
muon_px
nmuon_py
muon_py
nmuon_pz
muon_pz
nmuon_eta
muon_eta
nmuon_phi
muon_phi
nmuon_ch
muon_ch
nmuon_isSoft
muon_isSoft
nmuon_isTight
muon_isTight
nmuon_dxy
muon_dxy
nmuon_dz
muon_dz
nmuon_dxyError
muon_dxyError
nmuon_dzError
muon_dzError
nmuon_pfreliso03all
muon_pfreliso03all
nmuon_pfreliso04all
muon_pfreliso04all
nmuon_jetidx
muon_jetidx
nmuon_genpartidx
muon_genpartidx


By default, uproot treats all of the muon branches as distinct fields with distinct data. The <code>BaseSchema</code> within NanoEvents functions similarly. But this data is not independent! For one, we would expect the size of all of the <code>muon_*</code> subarrays to be the same (each muon has one of each field, each event has the same number of muons). Using a NanoEvents schema, we can clean our data up a little bit by nesting all of the muon fields beneath a common <code>muon</code> object. This is precisely what we have done in our schema:

* Get the schema working, then continue here

In [72]:
#events = factory with new schema

In [53]:
#events.fields

In [54]:
#events.Muon.fields

We can, of course, do similar nesting to other fields within our events array. This will make the surface-level view more compact and navigable. But there are also other benefits when we turn our attention to another feature of schemas: physics methods. We now have an <code>events.muon</code> object. Nominally, it serves as a category for our nesting structure, but it would make physical sense to treat it as a LorentzVector. This behavior has indeed been built into our new schema:

In [73]:
#(events.muon[0] + events.muon[1]).pt

One other implication of this feature is that, in addition to the fields explicitly mentioned under the MuonArray, we can also access other LorentzVector formulations with NanoEvents:

In [74]:
#events.muon.x, events.muon.y, events.muon.z, events.muon.mass

And we have the standard LorentzVector methods defined, such as delta_r for computing the distance between two four-vectors:

In [75]:
#events.muon[0].delta_r(events.muon[1])

** If possible, we should add examples of cross-references here.

## **Hists**

NanoEvents has given us data which we can access with the appropriate physics imposed. It's only natural that we now move on to plotting that data. Let's consider everybody's favorite simple example: plotting the mass of the Z boson based on the dilepton mass of opposite-charge, same-flavor lepton pairs. This should give us a peak at ~91.12, as many such pairs result from Z decays.

In [26]:
# Since they're same-flavor, we can just find dimuons and dielectrons independently. Make sure they're opposite charge and that each event has two leptons.
#dimuons = events.muon[(ak.num(events.muon, axis=1) == 2) & (ak.sum(events.muon.charge, axis=1) == 0)]
#dielectrons = events.electron[(ak.num(events.electron, axis=1) == 2) & (ak.sum(events.electron.charge, axis=1) == 0)]

Our <code>dimuons</code> array should now contain only opposite-charge muon pairs and our <code>dielectrons</code> opposite-charge electron pairs. Let's check!

In [76]:
#dimuons, dielectrons

Note that the masks performed a cut at the event level rather than the muon level. We have fewer events, but the same amount of leptons in each event (in the events that we kept). That means we've lost the connection between muons and electrons - they've been downselected and it is not the case that the 1st subarray in our electron array is the same event as the 1st subarray in our muon array. This isn't a problem since we're handling the two independently, but there are ways to handle such a selection without downselection if such indexing needs to be preserved.

All we need now is the dilepton mass. Awkward arrays can be indexed in a similar way as numpy array, so <code>dimuons[:, 0]</code> will select the first muon in every <code>dimuon</code> event. Recall that NanoEvents allows us to treat mathematical operations on the muon array level as LorentzVector objects. The same goes for our electrons array, of course. That makes our life easy:

In [77]:
#mumu_mass = (dimuons[:, 0] + dimuons[:, 1]).mass
#ee_mass = (dielectrons[:, 0] + dielectrons[:, 1]).mass

#mumu_mass, ee_mass

We've effectively collapsed our subarrays by finding the mass of the pairs, so now we have a flat array. It is of the same size (respectively) as our dimuons and dielectrons arrays above. We now have our data!

What else do we need for plotting? Well, a histogram is essentially a way to reduce our data. We can't just plot every value of dimuon mass, so we divide up our range of masses into n bins across some reasonable range. Thus, we need to define the mapping for our reduction; defining the number of bins and the range is sufficient for this.

In our case, let's plot 50 bins between values of 20 and 150. This removes anomalously low-mass data at the lower end, and merely cuts off a shrinking tail on the higher end. Because a histogram can contain an arbitrary amount of bins (in other words, dimensions), we also need to give our bin a label (which becomes its reference in our code) and a name (which is the name of the axis that users see when the histogram is plotted).

In [79]:
#ll_bin = hist.Bin(label="dilep_mass", name="$Dilepton Mass$", n_or_arr=50, lo=20, hi=150)

We are still not *yet* ready to plot. We have two masses we'd like to plot, and it doesn't make much sense to throw ee masses into the same bins as $\mu\mu$ masses. We want to keep these separate. We do so by introducing a categorical axis. Another example of when we might use a categorical axis is to keep data from different datasets separate, though in our case we're only working with a single dataset.

The definition of a categorical axis follows that of the bin axis, though this time we don't need any sort of reduction. Categories are specified at the point of filling. 

In [47]:
#ll_cat = hist.Cat("lepton", "Lepton Flavor")

We finally have all of the ingredients needed for a histogram! All that remains is to mix them together:

In [48]:
#ll_hist = hist.Hist("Counts", ll_cat, ll_bin)

Fill it with our data:

In [49]:
#ll_hist.fill(lepton="$\mu\mu$", dilep_mass=mumu_mass)
#ll_hist.fill(lepton="ee", dilep_mass=ee_mass)

And plot!

In [80]:
#hist.plot1d(ll_hist)

## **Processors**

## Appendix

### Lookup Tables

### ServiceX

## Acknowledgements